In [ ]:
import requests
import os
import asyncio
from datetime import datetime, timedelta
from IPython.display import display, update_display

In [ ]:
api_key = os.getenv("SYSTEMLINK_API_KEY")
http_url = os.getenv('SYSTEMLINK_HTTP_URI')

In [ ]:
session = requests.Session()
session.headers.update({
    "Content-Type": "application/json",
    'x-ni-api-key': api_key,
})

In [ ]:
create_execution_endpoint = f"{http_url}/ninbexecution/v1/executions"
info_about_execution_endpoint = f"{http_url}/ninbexecution/v1/executions/"
query_notebook_endpoint = f"{http_url}/ninotebook/v1/notebook/query"
auth_api_key_endpoint = f"{http_url}/niauth/v1/auth"


In [ ]:
def split_dates(start_date: str, end_date: str):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    result = []
    current = start
    while current <= end:
        for_date = current.strftime("%Y-%m-%d")
        result.append({
            "for_date": for_date
        })
        current += timedelta(days=1)
    return result

In [ ]:
def get_workspace_id():
    response = session.get(url=auth_api_key_endpoint)
    response.raise_for_status()
    data = response.json()
    
    workspaces = data["workspaces"]
    return next((w["id"] for w in workspaces if w.get("default") is True), workspaces[0]["id"])

In [ ]:
def get_notebook_id():
    payload={
        "filter":"name.StartsWith(\"Utilization to Tags SLE\")",
        "take":1
    }
    response = session.post(url=query_notebook_endpoint, json=payload)
    response.raise_for_status()
    data = response.json()
    if (data["notebooks"]):
        return data["notebooks"][0]["id"]
    else:
        raise Exception("Publish Utilization to Tags SLE script")

In [ ]:
def create_execution(notebookId, for_date, target_workspaces=[]):
    payload=[{
        "notebookId":notebookId,
        "parameters":{
            "target_date":for_date,
            "target_workspaces": target_workspaces
        },
        "workspaceId": workspace_id,
        "resultCachePeriod": 0,
    }]
    response = session.post(url=create_execution_endpoint, json=payload)
    response.raise_for_status()
    data = response.json()
    print(f"Execution id - {data["executions"][0]["id"]}")
    return data["executions"][0]["id"]

In [ ]:
async def check_execution(execution_id, date):
    handle = display("", display_id=True)

    while True:
        await asyncio.sleep(1)

        response = session.get(url=info_about_execution_endpoint + execution_id)
        response.raise_for_status()
        data = response.json()

        status = str(data.get("status", "")).strip()

        if status in {"FAILED", "CANCELED", "TIMED_OUT"}:
            update_display(f"❌ {date} - {status}", display_id=handle.display_id)
            raise Exception(data.get("exception") or f"Execution {execution_id} failed with {status}")

        elif status == "SUCCEEDED":
            update_display(f"✅ {date} - {status}", display_id=handle.display_id)
            return True

        else:
            update_display(f"⏳ {date} - {status}", display_id=handle.display_id)


In [ ]:
notebook_id = get_notebook_id()
workspace_id = get_workspace_id()

# Usage

In [ ]:
dates = split_dates("2025-09-20", "2025-09-27")
target_workspaces = ["a99d186f-8e2b-499c-9707-a30e2174c007"]
for date in dates:
    execution_id = create_execution(notebook_id, date["for_date"], target_workspaces)
    status = await check_execution(execution_id, date["for_date"])